In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv(r'/kaggle/input/playground-series-s4e2/train.csv')
df_test = pd.read_csv(r'/kaggle/input/playground-series-s4e2/test.csv')

# Explore Data

In [4]:
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [5]:
df_train.shape

(20758, 18)

In [6]:
df_train.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [8]:
df_train.isna().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [9]:
df_train.duplicated().sum()

0

In [10]:
df_train.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


# Transform Train Data 

In [11]:
la = LabelEncoder()

In [12]:
df_obj = df_train.select_dtypes(include = 'object')
df_NonObj = df_train.select_dtypes(exclude = 'object')

In [13]:
df_obj

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad
0,Male,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,yes,yes,Frequently,no,no,no,Automobile,Normal_Weight
2,Female,yes,yes,Sometimes,no,no,no,Public_Transportation,Insufficient_Weight
3,Female,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...
20753,Male,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation,Obesity_Type_II
20754,Male,no,yes,Frequently,no,no,Sometimes,Public_Transportation,Insufficient_Weight
20755,Male,yes,yes,Sometimes,no,no,no,Public_Transportation,Obesity_Type_II
20756,Male,yes,yes,Sometimes,no,no,no,Automobile,Overweight_Level_II


In [14]:
df_NonObj

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473
1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000
2,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584
3,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199
4,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...
20753,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680
20754,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000
20755,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439
20756,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834


In [15]:
for i in range (0,df_obj.shape[1]):
    df_obj.iloc[:,i]= la.fit_transform(df_obj.iloc[:,i])

In [16]:
df_obj

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad
0,1,1,1,2,0,0,1,3,6
1,0,1,1,1,0,0,2,0,1
2,0,1,1,2,0,0,2,3,0
3,0,1,1,2,0,0,1,3,4
4,1,1,1,2,0,0,1,3,6
...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,3
20754,1,0,1,1,0,0,1,3,0
20755,1,1,1,2,0,0,2,3,3
20756,1,1,1,2,0,0,2,0,6


In [17]:
df_obj = df_obj.astype('int')

In [18]:
data = pd.concat([df_obj,df_NonObj], axis=1)
data

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,1,3,6,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473
1,0,1,1,1,0,0,2,0,1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000
2,0,1,1,2,0,0,2,3,0,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584
3,0,1,1,2,0,0,1,3,4,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199
4,1,1,1,2,0,0,1,3,6,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,3,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680
20754,1,0,1,1,0,0,1,3,0,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000
20755,1,1,1,2,0,0,2,3,3,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439
20756,1,1,1,2,0,0,2,0,6,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834


# Transform Test Data

In [19]:
df_obj = df_test.select_dtypes(include = 'object')
df_NonObj = df_test.select_dtypes(exclude = 'object')
for i in range (0,df_obj.shape[1]):
    df_obj.iloc[:,i]= la.fit_transform(df_obj.iloc[:,i])
df_obj = df_obj.astype('int')
test = pd.concat([df_obj,df_NonObj], axis=1)
test

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,2,3,20758,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000
1,0,1,1,2,0,0,2,3,20759,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000
2,0,1,1,2,0,0,2,3,20760,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502
3,1,1,1,2,0,0,2,3,20761,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000
4,0,1,1,2,0,0,2,3,20762,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,1,1,0,2,0,0,2,3,34593,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632
13836,0,0,1,2,0,0,2,3,34594,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000
13837,0,0,1,1,0,0,2,3,34595,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000
13838,1,1,1,2,0,0,3,3,34596,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000


### Use StandardScaler

In [20]:
sc = StandardScaler()

In [21]:
scal1 = data[['Age']]
scal2 = data[['Weight']]
data['Age']= sc.fit_transform(scal1)
data['Weight']= sc.fit_transform(scal2)

In [22]:
data

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,1,3,6,0,0.105699,1.699998,-0.235713,2.000000,2.983297,2.763573,0.000000,0.976473
1,0,1,1,1,0,0,2,0,1,1,-1.027052,1.560000,-1.170931,2.000000,3.000000,2.000000,1.000000,1.000000
2,0,1,1,2,0,0,2,3,0,2,-1.027052,1.711460,-1.430012,1.880534,1.411685,1.910378,0.866045,1.673584
3,0,1,1,2,0,0,1,3,4,3,-0.507929,1.710730,1.644770,3.000000,3.000000,1.674061,1.467863,0.780199
4,1,1,1,2,0,0,1,3,6,4,1.371197,1.914186,0.224054,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,3,20753,0.227725,1.766626,0.996987,2.919584,3.000000,2.151809,1.330519,0.196680
20754,1,0,1,1,0,0,1,3,0,20754,-1.027052,1.710000,-1.436296,3.000000,4.000000,1.000000,2.000000,1.000000
20755,1,1,1,2,0,0,2,3,3,20755,-0.657669,1.819557,0.670717,2.407817,3.000000,2.000000,1.158040,1.198439
20756,1,1,1,2,0,0,2,0,6,20756,1.760067,1.700000,-0.165574,2.671238,1.971472,2.144838,0.000000,0.973834


In [23]:
scal1 = test[['Age']]
scal2 =test[['Weight']]
test['Age']= sc.fit_transform(scal1)
test['Weight']= sc.fit_transform(scal2)

In [24]:
test

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,2,3,20758,0.508163,1.848294,1.273786,2.938616,3.000000,2.825629,0.855400,0.000000
1,0,1,1,2,0,0,2,3,20759,-0.509128,1.600000,-0.818988,2.000000,1.000000,3.000000,1.000000,0.000000
2,0,1,1,2,0,0,2,3,20760,0.353000,1.643355,0.927432,3.000000,3.000000,2.621877,0.000000,0.250502
3,1,1,1,2,0,0,2,3,20761,-0.512705,1.553127,0.623672,2.000000,2.977909,2.786417,0.094851,0.000000
4,0,1,1,2,0,0,2,3,20762,0.353000,1.627396,0.668336,3.000000,3.000000,2.653531,0.000000,0.741069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,1,1,0,2,0,0,2,3,34593,-0.107749,1.721384,-0.358246,2.813234,3.000000,1.000000,0.807076,0.778632
13836,0,0,1,2,0,0,2,3,34594,0.870277,1.590000,-0.972181,3.000000,3.000000,2.000000,0.000000,0.000000
13837,0,0,1,1,0,0,2,3,34595,-0.175379,1.585547,-1.647125,3.000000,2.273740,2.000000,1.949840,1.000000
13838,1,1,1,2,0,0,3,3,34596,-0.509128,1.620000,-1.316865,2.000000,3.000000,2.000000,3.000000,2.000000


# Creat Model

In [25]:
x = data.drop(['NObeyesdad','id'], axis =1)

In [26]:
y = data['NObeyesdad']

In [27]:
x_train , x_test, y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=21)

In [28]:
model1 = LogisticRegression()
model2 = RandomForestClassifier()
model3 = GaussianNB()
model4 = XGBClassifier()
model5 = GradientBoostingClassifier()
model6 = SVC()

In [29]:
def pred(model):
    model.fit(x_train, y_train)
    pre = model.predict(x_test)
    print(classification_report(pre , y_test))

In [30]:
pred(model1)

              precision    recall  f1-score   support

           0       0.86      0.79      0.82       560
           1       0.72      0.71      0.71       642
           2       0.78      0.81      0.79       574
           3       0.96      0.92      0.94       634
           4       1.00      0.99      1.00       823
           5       0.58      0.67      0.62       418
           6       0.63      0.62      0.63       501

    accuracy                           0.81      4152
   macro avg       0.79      0.79      0.79      4152
weighted avg       0.81      0.81      0.81      4152



In [31]:
pred(model2)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       518
           1       0.90      0.85      0.87       668
           2       0.88      0.90      0.89       583
           3       0.97      0.97      0.97       610
           4       1.00      1.00      1.00       820
           5       0.74      0.84      0.78       427
           6       0.85      0.79      0.82       526

    accuracy                           0.90      4152
   macro avg       0.89      0.90      0.89      4152
weighted avg       0.91      0.90      0.90      4152



In [32]:
pred(model3)

              precision    recall  f1-score   support

           0       0.88      0.69      0.77       668
           1       0.47      0.64      0.54       459
           2       0.60      0.40      0.48       893
           3       0.93      0.69      0.79       826
           4       1.00      0.96      0.98       853
           5       0.27      0.61      0.38       217
           6       0.26      0.53      0.35       236

    accuracy                           0.66      4152
   macro avg       0.63      0.65      0.61      4152
weighted avg       0.74      0.66      0.69      4152



In [33]:
pred(model4)

              precision    recall  f1-score   support

           0       0.93      0.92      0.93       527
           1       0.89      0.86      0.88       648
           2       0.88      0.91      0.90       578
           3       0.98      0.96      0.97       614
           4       1.00      1.00      1.00       821
           5       0.77      0.83      0.80       447
           6       0.86      0.82      0.84       517

    accuracy                           0.91      4152
   macro avg       0.90      0.90      0.90      4152
weighted avg       0.91      0.91      0.91      4152



In [34]:
pred(model5)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       526
           1       0.90      0.87      0.88       650
           2       0.88      0.91      0.89       577
           3       0.97      0.96      0.97       612
           4       1.00      1.00      1.00       819
           5       0.78      0.84      0.81       449
           6       0.84      0.80      0.82       519

    accuracy                           0.91      4152
   macro avg       0.90      0.90      0.90      4152
weighted avg       0.91      0.91      0.91      4152



In [35]:
pred(model6)

              precision    recall  f1-score   support

           0       0.93      0.85      0.89       570
           1       0.78      0.81      0.79       610
           2       0.82      0.85      0.83       580
           3       0.97      0.95      0.96       618
           4       1.00      1.00      1.00       820
           5       0.63      0.72      0.67       422
           6       0.74      0.68      0.71       532

    accuracy                           0.85      4152
   macro avg       0.84      0.84      0.84      4152
weighted avg       0.86      0.85      0.85      4152



In [36]:
testx=test.drop('id',axis=1)

In [37]:
prex=model5.predict(testx)

In [38]:
submission=pd.DataFrame({"id":test['id'],"NObeyesdad":prex})
submission

,id,NObeyesdad
0,20758,3
1,20759,5
2,20760,4
3,20761,2
4,20762,4
...,...,...
13835,34593,6
13836,34594,1
13837,34595,0
13838,34596,1


In [39]:
la.fit(prex)
submission['NObeyesdad'] = la.inverse_transform(prex)